## [문화역세권 분석](https://www.culture.go.kr/bigdata/user/sjuvisual/sju_visualSub.do?sjuP=/visual/page01)
- [문화역세권 데이터](https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=f41966cf-a563-40da-875c-5c317faee505)
- [공공문화역세권 데이터](https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=a8feb92a-1c97-4f2e-8faa-7d3fa51d7a45)

### Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# Plotly 설치방법: Anaconda Prompt 창에서 아래 커맨드를 실행 
# pip install plotly

In [ ]:
import plotly.express as px

## 1. 전국 문화시설 수

### Read CSV file

In [ ]:
df = pd.read_csv('data/01_문화역세권분석.csv')
df

### Edit columns

In [ ]:
df.rename(columns={'id': '고유ID', 'lclas': 'L1 대분류', 'mlsfc': 'L2 중분류', 'fclt_name': '시설명', 
                   'ctprvn_nm': '시도', 'sgnr_nm': '시군구', 'legaldong_cd': '법정동코드', 
                   'legalemd_nm': '법정읍면동명', 'adstrd_cd': '행정동코드', 
                   'adstrd_nm': '행정동명', 'rdnmaddr_cd': '도로명주소코드', 
                   'rdnm_addr': '도로명주소', 'zip_cd': '우편번호', 'grid_cd': '격자코드', 
                   'x_cd': 'X좌표', 'y_cd': 'Y좌표', 'buld_totar': '시설현황 건물 연면적', 
                   'usemem_nm': '연간이용자수', 'infl_factor': '영향도지수', 
                   'file_name': '파일명', 'base_ymd': '기준일'}, inplace=True)
                    

In [ ]:
df.drop(['고유ID', '시설명', '법정동코드', '행정동코드', '도로명주소코드', '격자코드',  '파일명', '기준일'], axis=1, inplace=True)
df

### Information

In [ ]:
df['시도'].value_counts()

In [ ]:
df['L2 중분류'].value_counts()

### Groupby

In [ ]:
gp_cnt = df.groupby('시도').size().reset_index()
gp_cnt.rename(columns={0:'문화시설수'}, inplace=True)

In [ ]:
gp_cnt

### Visualization

In [ ]:
fig = px.bar(gp_cnt, x='시도', y='문화시설수', color='시도', title='전국 문화시설 수')
fig.update_xaxes(categoryorder = 'total descending')
fig.show()

## 2. 전국 문화시설 수 - 분류별

In [ ]:
gp_cnt_cat = df.groupby(['시도','L2 중분류']).size().reset_index()
gp_cnt_cat.rename(columns={0:'문화시설수'}, inplace=True)

In [ ]:
gp_cnt_cat

In [ ]:
fig = px.bar(gp_cnt_cat, x='시도', y='문화시설수', color='L2 중분류', barmode='group', title ='전국 문화시설 수 - 분류별')
fig.update_xaxes(categoryorder = 'total descending')
fig.show()

## 3. 전국 문화시설  영향도 지수

In [ ]:
gp_eff = df.groupby('시도').mean()['영향도지수'].reset_index()

In [ ]:
gp_eff

In [ ]:
fig = px.bar(gp_eff, x='시도', y='영향도지수', color='시도', title='지역별 시설 영향도 지수 평균')
fig.update_xaxes(categoryorder = 'total descending')
fig.show()

## 4. 전국 문화시설 위치

In [ ]:
fig = px.scatter_mapbox(df, lat='X좌표', lon='Y좌표', color='시도', hover_name='L2 중분류',
                        color_continuous_scale=px.colors.cyclical.IceFire, size_max=400, zoom=6)
fig.update_layout(mapbox_style = 'open-street-map')
fig.show()

## 5. 인구 대비 전국 문화시설 수

In [ ]:
df_p = pd.read_excel('data/05_인구.xlsx')
df_p.set_index('지역', inplace=True)
df_p

In [ ]:
df_p['문화시설수'] =  df.groupby('시도').size()

In [ ]:
df_p['시설수/10만명'] = (df_p['문화시설수'] * 100000 / df_p['인구']).round(2)

In [ ]:
df_p

In [ ]:
fig = px.bar(df_p, y='시설수/10만명', color=df_p.index, title='인구 10만명 당 문화시설 수')
fig.update_xaxes(categoryorder = 'total descending')
fig.show()

## 6. 공공문화역세권 분석
- [공공문화역세권 데이터](https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=a8feb92a-1c97-4f2e-8faa-7d3fa51d7a45)

In [ ]:
dfpub = pd.read_csv('data/02_공공문화역세권.csv')
dfpub

In [ ]:
dfpub.rename(columns={'CTPRVN_NM': '시도', 'SGNR_NM': '시군구', 'flag_pub_cnt': '공공문화시설수',
                      'flag_priv_cnt': '민간문화시설수', 'flag_pub_priv_cnt': '전체문화시설수',
                      'flag_pub_rate': '공급률', 'flag_pub_percent': '공급률(%)',
                      'FILE_NAME': '파일명', 'BASE_YMD': '기준일'}, inplace=True)
dfpub.drop(['공급률', '파일명', '기준일'], axis=1, inplace=True)
dfpub

In [ ]:
gp_pub = dfpub.groupby('시도').agg({'공공문화시설수':'sum', '민간문화시설수':'sum', '전체문화시설수':'sum', '공급률(%)':'mean'})
gp_pub.reset_index(inplace=True)

In [ ]:
gp_pub

In [ ]:
gp_pub = pd.melt(gp_pub, id_vars=['시도'])

In [ ]:
gp_pub

In [ ]:
fig = px.bar(gp_pub, x='시도', y='value', color='variable', barmode='group', title = '전국 문화시설 수')
fig.update_xaxes(categoryorder = 'total descending')
fig.show()

---

In [ ]:
# End of file